In [1]:
# PyTorch 101, Part 1: Understanding Graphs, Automatic Differentiation and Autograd
# https://blog.paperspace.com/pytorch-101-understanding-graphs-and-automatic-differentiation/

#  PyTorch's Autograd engine performs automatic differentiation.

# 학습목표 
# 1. 그래프 이해 
# 2. 자동미분 과 autograd 이해

In [2]:
# computation graph

# 그래프의 구조와 상관없이 균일하게 미분가능
# node 들은 사용자 정의 변수 말고는 다 연산자이다. 

# 여기서 leaf node 란 수학적 연산과정에 의해 생선된 것들이 아니라
# 이름 그자체로 대표되는 것들이다. 

# leaf node 를 제외한 것들은 
# 함수의 결과값으로 생각해도 상관이 없다. 

In [1]:
# Pytorch Autograd 

# pytorch 가 computation graph 를 생성하고 싶으면 
# requires_grad = True 로 설정하여야 한다. 

# 특징
# 1. requires_grad => contagious 하다. 
# (대충 True 로 계산된 애들의 결과값 역시 그렇게 댐)

# 2. grad_fn
# 위 변수를 만드는데 사용되었던 연산을 저장해준다.
# 물론 requires_grad =False 일시 None 이다. 왜냐하면
# computation graph 자체를 생성하지 않기 때문이다. 

# leaf node 의 grad_fn 은 None 이다! 기억하자 
# (당연한거임 역전파가 끝나는 지점이기에..)

In [3]:
import torch 

a = torch.randn((3,3), requires_grad = True)

w1 = torch.randn((3,3), requires_grad = True)
w2 = torch.randn((3,3), requires_grad = True)
w3 = torch.randn((3,3), requires_grad = True)
w4 = torch.randn((3,3), requires_grad = True)

b = w1*a 
c = w2*a

d = w3*b + w4*c 

L = 10 - d
print("The grad fn for a is", a.grad_fn)
print("The grad fn for d is", d.grad_fn)

True
The grad fn for a is None
The grad fn for d is <AddBackward0 object at 0x000001BA88F9C0A0>


In [4]:
# pytorch 함수 두개 

# 1. 순전파 -> 쉬우니 걍 넘어감

# 2. 역전파 -> 관여한 변수들에 대해서 기울기 다 구해버림
# 여러칸 건너띈것들은 편미분, chain rule 사용해서 다 기울기 구해버림

# backward 함수는 기울기 구하는 것에 끝난다고 이해하는것이 맞는것 같다.
# 기울기로 업데이트를 할려면 optim 을 활용하는 것이 맞다. 
# 혹은 직접 빼줘야 한다.

In [5]:
# back propagtion 주의 사항

# backward() 는 vector 단위에서는 실행이 안된다.

import torch 

a = torch.randn((3,3), requires_grad = True)

w1 = torch.randn((3,3), requires_grad = True)
w2 = torch.randn((3,3), requires_grad = True)
w3 = torch.randn((3,3), requires_grad = True)
w4 = torch.randn((3,3), requires_grad = True)

b = w1*a 
c = w2*a

d = w3*b + w4*c 

# 방법1
L = (10 - d).sum() # scalar 로 바꾸어 표현해 주어야함!

# 방법2
# Replace L.backward() with 
# L.backward(torch.ones(L.shape))

L.backward()

In [7]:
# Pytorch 의 그래프가 Tensorflow 의 그래프와 다른이유

# Pytorch -> Dynamic Computation Graph
# 특징
# 1. 그래프가 즉석에서 생성됨
# 2. forward 과정, 연산이 진행되기 전까지 그래프 생성 안됨

a = torch.randn((3,3), requires_grad = True)   #No graph yet, as a is a leaf

w1 = torch.randn((3,3), requires_grad = True)  #Same logic as above

b = w1*a   #Graph with node `mulBackward` is created.

tensor([[-1.8171,  0.4143,  1.7057],
        [-2.3509,  0.2452,  0.0339],
        [-0.5116,  0.7795,  0.1405]])

In [ ]:
# 개인적인 생각으로는 
# leaf 와 non-leaf 의 목적은 
# 생성된 그래프가 미분을 더 하느냐 마느냐의 차이로 판가름하기위해
# is_leaf 와 non_leaf 를 판별한다고 생각한다. 


# 그런다음 non-leaf 노드의 버퍼들이 그래프와 중간값들을 위해서 할당된다.(나중에 그레이디언트 계산에 사용됨). 
# 그레이디언트를 계산할 때 역으로 호출하면, 이 버퍼들(non-leaf 의 변수들)은 본질적으로 자유로워지고 
# 그래프는 파괴된다(어떤 의미에서는 그레이디언트를 계산하기 위한 값을 보유하는 버퍼가 사라졌기 때문에
#  이 버퍼를 통해 역전파할 수 없다).

# pytorch 의 retain 함수가 추후 이거와 관련이 있다고 생각한다.
